с человеческим обучением разобрался.
я там беру данные, самостоятельно анализирую, ищу закономерности и придумываю правила с помощью которых можно определить стоимость автомобиля.

машинное обучение делает тоже самое, оно позволяет определять целевую переменную по фичам(признакам) которые имеем
машинное обучение нахоодит эти правила автоматически. Я должен только загрузить данные и проследить, что машина справилась. Но сам процесс поиска связей машина делает сама. Она смотрит данные и пытается найти какие то закономерности.

машина обучается на данных. Она смотрит на примерах и пытается понять какие то правила.

буду работать на catboost

не надо вникать как он работает. сначала надо научиться пользоваться. решать прикладные задачи.


не надо вникать что такое дерево решений и тд. для меня сейчас catboost это как машина с коробкой автомат
просто сел и поехал. все супер интуитивно и понятно.
catboost дает крутые результаты без тюнинга

на текущий момент градиентный бустинг, это самый эффективный алгоритм для работы с табличными данными

1. [imports and separate data](#imports-and-separate-data) 
2. [feature list](#feature-list)
3. [cat boost first start](#cat-boost-first-start)
4. [all feautures learning](#all-feautures-learning)
5. [another parametrs](#another-parametrs)
6. [learning on all data](#learning-on-all-data)
7. [regression metrics](#regression-metrics)


# imports and separate data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('uk-used-cars/bmw.csv')
df.keys(), df.columns

(Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
        'mpg', 'engineSize'],
       dtype='object'),
 Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
        'mpg', 'engineSize'],
       dtype='object'))

In [3]:
# Обучающая и тестовая выборка. потом нужно будет проверить. как catboost обучился
#60/20/20 train, val, test set
# делаю для теста пока 40 процентов
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size=0.6, random_state=42)

для обучения катбуст, помимо тех двух выборок(тестовая и тренирровачная) нужна еще и валидационная выборка.
кат буст обучается итерациями(несколько сот раз) и каждый раз он извлекает какие то взаимосвязи. и каждый раз он извлекает все больше и больше каких то данных. и находит какие то зависимости между фичами(переменными) и целевым признаком. и проблема в том, если не остановить процесс, то катбуст просто вызубрит эту выборку, он запомнит наизусть. Он просто вызубрил и запомнил выборку, и не понял паттерны и зависимости.
что бы этого измежать используется валидационная выборка

In [4]:
# и делю тестовые данные пополам(на валидационные и тестовые)
val, test = train_test_split(test, train_size=0.5, random_state=42)

In [5]:
len(val) / len(df)

0.19998144884519062

In [6]:
len(test) / len(df)

0.20007420461923756

# feature list 

подготовить список фичей и признаков, на которых будем обучать катбуст.
смотрю названия колонок, но там есть и целевой признак "price", и его нужно убрать от тудава

фичи кладу в X(большая буква)

таргет кладу в y(маленькая буква)

в данных будут числовые и категориальные фичи. и для катбуста важно указать те столбцы где указан текст.
и что бы их использовать их сначала нужно преобразовать в числовые признаки.
для градиентного бустинга нужны только числа.

catboost сам все преобразует. (категориальные фичи в числовые данные)
категориальные(текст) в нашем случае это transmission, fuelType, model

иногда может быть такое что вроде бы числовое значение в столбце, а оно на самом деле категориальное. 

типа например color и там в строчках 1,2,3,2,1

и такие тоже надо преобразовывать(так как они на самом деле категориальные)

In [7]:
train.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [8]:
train

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5133,X6,2015,25000,Semi-Auto,26100,Diesel,205,45.6,3.0
2121,X2,2019,31980,Automatic,4804,Petrol,145,34.0,2.0
6576,X2,2018,22995,Semi-Auto,24136,Petrol,145,38.7,2.0
8330,3 Series,2016,12999,Manual,52224,Diesel,125,62.8,2.0
3161,X3,2019,32980,Semi-Auto,1961,Diesel,150,54.3,2.0
...,...,...,...,...,...,...,...,...,...
5734,3 Series,2018,17995,Semi-Auto,40456,Diesel,145,64.2,2.0
5191,1 Series,2016,19950,Semi-Auto,23210,Petrol,200,39.8,3.0
5390,X2,2019,25450,Semi-Auto,5652,Diesel,145,47.9,2.0
860,X1,2016,19850,Semi-Auto,14542,Diesel,125,60.1,2.0


In [9]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price'] 

# принцип преобразования категориальных фичей

я думаю сделать словарь для категориальных переменных. И это не правильно

категориальные переменные в классическом машиннном обучении преобразуют с помощью ONE HOT ENCODING

In [7]:
# только для transmission появилось три столбца. И там либо 0(False) либо 1(True)
pd.get_dummies(train['transmission']) 

,Automatic,Manual,Semi-Auto
5133,False,False,True
2121,True,False,False
6576,False,False,True
8330,False,True,False
3161,False,False,True
...,...,...,...
5734,False,False,True
5191,False,False,True
5390,False,False,True
860,False,False,True


In [8]:
train['transmission']

5133    Semi-Auto
2121    Automatic
6576    Semi-Auto
8330       Manual
3161    Semi-Auto
          ...    
5734    Semi-Auto
5191    Semi-Auto
5390    Semi-Auto
860     Semi-Auto
7270    Automatic
Name: transmission, Length: 6468, dtype: object

Когда используется One-Hot Encoding, каждая категория преобразуется в отдельный бинарный столбец, как в нашем примере с колонками Automatic, Manual, Semi-Auto. Поскольку эти столбцы являются взаимно исключающимися, для каждой строки всегда только один столбец будет равен True (1), а остальные будут равны False (0).

Идея заключается в том, что можно удалить один из столбцов, потому что его информация может быть однозначно восстановлена из других столбцов. Например, если в строке оба столбца Automatic и Semi-Auto равны False, это автоматически означает, что Manual должен быть True. Таким образом, удаление одного столбца не приведет к потере информации.

Это помогает избежать избыточности в данных и сократить количество столбцов, что может ускорить работу некоторых моделей и уменьшить размер данных.


сначала надо заменить Nan на какое-нибудь "no_info" например, и потом делать OHE. Так же можно поэкспериментировать и заполнить пропуски самым часто встречающимся значением и посмотреть как измениться качество модели.











# cat boost first start

cat - это не кошка. это category. так как он хорошо работает с категориальными фичами.

In [14]:
# установка катбуст
#! pip install catboost

беру CatBoostRegressor потому что беру задачу регрессии. задача регрессии тогда когда предсказываем 
непрерывный таргет.(цена может принимать любое значение)


еще бывает задача классификации. когда нужно предсказыать конкретную категорю CatBoostClassifier

CatBoostRanker решает задачу ранжирования. когда нужно отранжировать элементы в порядке убывания или возрастания

In [9]:
from catboost import CatBoostRegressor # из него создам объект модели

In [10]:
X = ['year', 'transmission','engineSize']

cat_features = ['transmission']

y = ['price'] 

нужно указать какеи есть категориальные переменные

и еще попробую в первый раз запустить с теми фичами которые исользовал в человеческом обучениее
там были год, трансмиссия и двигатель

и дальше укажу метрику на которой буду считать ошибку на валидационной выборке

все доступные метрики тутова     https://catboost.ai/en/docs/references/custom-metric__supported-metrics

In [11]:
# по сути создали объект катбуста, указали какая метрика и указали категориальнные переменные
model = CatBoostRegressor(cat_features=cat_features, eval_metric='MAPE')

In [12]:
train[X]

,year,transmission,engineSize
5133,2015,Semi-Auto,3.0
2121,2019,Automatic,2.0
6576,2018,Semi-Auto,2.0
8330,2016,Manual,2.0
3161,2019,Semi-Auto,2.0
...,...,...,...
5734,2018,Semi-Auto,2.0
5191,2016,Semi-Auto,3.0
5390,2019,Semi-Auto,2.0
860,2016,Semi-Auto,2.0


In [92]:
# вызваю метод fit и указываю X и y (это названия столбцов из выборки)
# и так же надо указать валидационную выборку в скобках круглых, так же Х и у
model.fit(X=train[X], y=train[y], eval_set=(val[X], val[y]))

Learning rate set to 0.068263
0:	learn: 0.4607010	test: 0.4559908	best: 0.4559908 (0)	total: 3.96ms	remaining: 3.96s
1:	learn: 0.4360441	test: 0.4312870	best: 0.4312870 (1)	total: 7.26ms	remaining: 3.62s
2:	learn: 0.4165637	test: 0.4119748	best: 0.4119748 (2)	total: 11.3ms	remaining: 3.75s
3:	learn: 0.3960689	test: 0.3916895	best: 0.3916895 (3)	total: 16.3ms	remaining: 4.07s
4:	learn: 0.3788752	test: 0.3744255	best: 0.3744255 (4)	total: 19.7ms	remaining: 3.92s
5:	learn: 0.3617532	test: 0.3571447	best: 0.3571447 (5)	total: 23.1ms	remaining: 3.83s
6:	learn: 0.3468419	test: 0.3419874	best: 0.3419874 (6)	total: 26.7ms	remaining: 3.78s
7:	learn: 0.3321039	test: 0.3275046	best: 0.3275046 (7)	total: 30.3ms	remaining: 3.75s
8:	learn: 0.3186373	test: 0.3140204	best: 0.3140204 (8)	total: 34.9ms	remaining: 3.84s
9:	learn: 0.3062500	test: 0.3018196	best: 0.3018196 (9)	total: 39.5ms	remaining: 3.91s
10:	learn: 0.2963395	test: 0.2921336	best: 0.2921336 (10)	total: 42.9ms	remaining: 3.85s
11:	learn: 

178:	learn: 0.1533849	test: 0.1524557	best: 0.1524530 (177)	total: 193ms	remaining: 887ms
179:	learn: 0.1533652	test: 0.1524481	best: 0.1524481 (179)	total: 195ms	remaining: 886ms
180:	learn: 0.1533646	test: 0.1524482	best: 0.1524481 (179)	total: 195ms	remaining: 883ms
181:	learn: 0.1533523	test: 0.1524466	best: 0.1524466 (181)	total: 196ms	remaining: 882ms
182:	learn: 0.1533469	test: 0.1524426	best: 0.1524426 (182)	total: 197ms	remaining: 879ms
183:	learn: 0.1532932	test: 0.1524048	best: 0.1524048 (183)	total: 198ms	remaining: 879ms
184:	learn: 0.1532758	test: 0.1524100	best: 0.1524048 (183)	total: 199ms	remaining: 878ms
185:	learn: 0.1532710	test: 0.1524033	best: 0.1524033 (185)	total: 200ms	remaining: 876ms
186:	learn: 0.1532647	test: 0.1523994	best: 0.1523994 (186)	total: 201ms	remaining: 876ms
187:	learn: 0.1532477	test: 0.1523935	best: 0.1523935 (187)	total: 202ms	remaining: 874ms
188:	learn: 0.1532508	test: 0.1524030	best: 0.1523935 (187)	total: 203ms	remaining: 872ms
189:	learn

366:	learn: 0.1510400	test: 0.1519921	best: 0.1519481 (357)	total: 385ms	remaining: 663ms
367:	learn: 0.1510223	test: 0.1519787	best: 0.1519481 (357)	total: 386ms	remaining: 663ms
368:	learn: 0.1510180	test: 0.1519784	best: 0.1519481 (357)	total: 387ms	remaining: 662ms
369:	learn: 0.1510143	test: 0.1519845	best: 0.1519481 (357)	total: 388ms	remaining: 661ms
370:	learn: 0.1510058	test: 0.1519882	best: 0.1519481 (357)	total: 389ms	remaining: 660ms
371:	learn: 0.1509830	test: 0.1519881	best: 0.1519481 (357)	total: 391ms	remaining: 660ms
372:	learn: 0.1509777	test: 0.1519861	best: 0.1519481 (357)	total: 392ms	remaining: 659ms
373:	learn: 0.1509700	test: 0.1519974	best: 0.1519481 (357)	total: 393ms	remaining: 658ms
374:	learn: 0.1509649	test: 0.1519949	best: 0.1519481 (357)	total: 394ms	remaining: 657ms
375:	learn: 0.1509462	test: 0.1519901	best: 0.1519481 (357)	total: 395ms	remaining: 656ms
376:	learn: 0.1509333	test: 0.1519799	best: 0.1519481 (357)	total: 397ms	remaining: 655ms
377:	learn

547:	learn: 0.1498253	test: 0.1517089	best: 0.1516460 (514)	total: 578ms	remaining: 476ms
548:	learn: 0.1498188	test: 0.1517029	best: 0.1516460 (514)	total: 579ms	remaining: 475ms
549:	learn: 0.1498137	test: 0.1517043	best: 0.1516460 (514)	total: 580ms	remaining: 474ms
550:	learn: 0.1498084	test: 0.1516800	best: 0.1516460 (514)	total: 581ms	remaining: 473ms
551:	learn: 0.1497970	test: 0.1516725	best: 0.1516460 (514)	total: 582ms	remaining: 472ms
552:	learn: 0.1497944	test: 0.1516563	best: 0.1516460 (514)	total: 583ms	remaining: 471ms
553:	learn: 0.1497758	test: 0.1516609	best: 0.1516460 (514)	total: 584ms	remaining: 470ms
554:	learn: 0.1497716	test: 0.1516656	best: 0.1516460 (514)	total: 585ms	remaining: 469ms
555:	learn: 0.1497610	test: 0.1516578	best: 0.1516460 (514)	total: 586ms	remaining: 468ms
556:	learn: 0.1497569	test: 0.1516350	best: 0.1516350 (556)	total: 587ms	remaining: 467ms
557:	learn: 0.1497561	test: 0.1516377	best: 0.1516350 (556)	total: 587ms	remaining: 465ms
558:	learn

729:	learn: 0.1489447	test: 0.1514184	best: 0.1514104 (728)	total: 769ms	remaining: 285ms
730:	learn: 0.1489398	test: 0.1514236	best: 0.1514104 (728)	total: 771ms	remaining: 284ms
731:	learn: 0.1489182	test: 0.1514144	best: 0.1514104 (728)	total: 772ms	remaining: 283ms
732:	learn: 0.1489083	test: 0.1513998	best: 0.1513998 (732)	total: 773ms	remaining: 281ms
733:	learn: 0.1489037	test: 0.1513996	best: 0.1513996 (733)	total: 774ms	remaining: 280ms
734:	learn: 0.1489005	test: 0.1513996	best: 0.1513996 (733)	total: 775ms	remaining: 279ms
735:	learn: 0.1488997	test: 0.1514002	best: 0.1513996 (733)	total: 776ms	remaining: 278ms
736:	learn: 0.1488992	test: 0.1514015	best: 0.1513996 (733)	total: 777ms	remaining: 277ms
737:	learn: 0.1488906	test: 0.1513915	best: 0.1513915 (737)	total: 778ms	remaining: 276ms
738:	learn: 0.1488877	test: 0.1513921	best: 0.1513915 (737)	total: 779ms	remaining: 275ms
739:	learn: 0.1488826	test: 0.1514079	best: 0.1513915 (737)	total: 780ms	remaining: 274ms
740:	learn

916:	learn: 0.1482459	test: 0.1511865	best: 0.1511823 (914)	total: 962ms	remaining: 87.1ms
917:	learn: 0.1482442	test: 0.1511876	best: 0.1511823 (914)	total: 963ms	remaining: 86ms
918:	learn: 0.1482258	test: 0.1512017	best: 0.1511823 (914)	total: 964ms	remaining: 85ms
919:	learn: 0.1482256	test: 0.1512003	best: 0.1511823 (914)	total: 965ms	remaining: 83.9ms
920:	learn: 0.1482227	test: 0.1512008	best: 0.1511823 (914)	total: 966ms	remaining: 82.9ms
921:	learn: 0.1482206	test: 0.1512002	best: 0.1511823 (914)	total: 967ms	remaining: 81.8ms
922:	learn: 0.1482162	test: 0.1512168	best: 0.1511823 (914)	total: 969ms	remaining: 80.8ms
923:	learn: 0.1482121	test: 0.1512175	best: 0.1511823 (914)	total: 970ms	remaining: 79.7ms
924:	learn: 0.1482121	test: 0.1512167	best: 0.1511823 (914)	total: 971ms	remaining: 78.7ms
925:	learn: 0.1482134	test: 0.1512217	best: 0.1511823 (914)	total: 972ms	remaining: 77.7ms
926:	learn: 0.1482106	test: 0.1512212	best: 0.1511823 (914)	total: 973ms	remaining: 76.6ms
927

слева показана  ошибка на трейн, справа на тест, потом пишет лучший результат и в скобках номер лучшей итерации

в самом низу пишет номер лучшей итерации и лучший результат. после этой итерации уже не было улучшений

получилось bestTest = 0.1511823285
15 процентов

а было в человеческом обучении 16 процентов

In [13]:
# verbose=100 выводит каждую сотую итерацию, а не каждую просто
model.fit(X=train[X], y=train[y], eval_set=(val[X], val[y]), verbose=100)

Learning rate set to 0.068263
0:	learn: 0.4607010	test: 0.4559908	best: 0.4559908 (0)	total: 57ms	remaining: 56.9s
100:	learn: 0.1554098	test: 0.1538247	best: 0.1538247 (100)	total: 205ms	remaining: 1.82s
200:	learn: 0.1530463	test: 0.1523974	best: 0.1523588 (196)	total: 297ms	remaining: 1.18s
300:	learn: 0.1516833	test: 0.1522258	best: 0.1522198 (299)	total: 395ms	remaining: 918ms
400:	learn: 0.1507169	test: 0.1519076	best: 0.1519009 (398)	total: 498ms	remaining: 744ms
500:	learn: 0.1501075	test: 0.1516689	best: 0.1516502 (493)	total: 587ms	remaining: 585ms
600:	learn: 0.1494997	test: 0.1516109	best: 0.1515971 (597)	total: 693ms	remaining: 460ms
700:	learn: 0.1490643	test: 0.1514998	best: 0.1514998 (700)	total: 800ms	remaining: 341ms
800:	learn: 0.1486800	test: 0.1513459	best: 0.1513318 (762)	total: 901ms	remaining: 224ms
900:	learn: 0.1482781	test: 0.1512429	best: 0.1512429 (900)	total: 1s	remaining: 110ms
999:	learn: 0.1479669	test: 0.1512429	best: 0.1511823 (914)	total: 1.11s	remai


Предикт (predict) — это процесс, когда модель, обученная на данных, используется для предсказания значений на новых данных. Например, после того как модель обучилась на тренировочных данных, её применяют к тестовым данным для получения предсказаний (в данном случае, это могут быть прогнозы цен автомобилей).

Для чего он нужен: Предикт позволяет оценить, насколько хорошо модель справляется с задачей, сравнивая предсказанные значения с реальными (истинными) данными.

Что делать с предиктом: использовать эти предсказания для проверки точности модели или для дальнейшего анализа (например, вычисления ошибок предсказаний).

In [14]:
# делаем предикт. там в модели уже осталась лучшая итерация
model.predict(test[X])

array([45367.67756432, 21884.38898346, 16091.74678864, ...,
       28392.87755636, 16091.74678864, 27668.06069946])

In [15]:
test['price_pred'] = model.predict(test[X])

In [16]:
# проставили цены предсказанные в тестовую выборку. и дальше будем считать метрику
test

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,price_pred
4984,X6,2019,52990,Semi-Auto,3086,Diesel,145,34.9,3.0,45367.677564
4880,X1,2018,24081,Semi-Auto,13245,Diesel,150,60.1,2.0,21884.388983
9132,1 Series,2016,12999,Automatic,68949,Petrol,200,43.5,2.0,16091.746789
6451,1 Series,2020,11995,Semi-Auto,10,Petrol,150,34.5,2.0,32231.178149
7199,3 Series,2020,29875,Semi-Auto,150,Petrol,145,42.2,2.0,32231.178149
...,...,...,...,...,...,...,...,...,...,...
9627,3 Series,2015,14999,Automatic,78680,Diesel,160,52.3,3.0,19305.028412
6096,4 Series,2019,29676,Automatic,7365,Petrol,145,41.5,3.0,43991.055403
1747,5 Series,2019,30570,Automatic,3067,Hybrid,145,49.6,2.0,28392.877556
4023,3 Series,2016,14999,Automatic,70054,Hybrid,0,148.7,2.0,16091.746789


In [17]:
# Метрика поможет тебе понять, насколько точно модель предсказывает цену автомобилей.
# использую фукнкцию из человеческого обучения
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
def error(y_true, y_pred):
    print(mean_absolute_error(y_true=y_true, y_pred=y_pred))
    print(mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred))

In [18]:
error(y_true=test['price'], y_pred=test['price_pred'])

3345.7246072100406
0.15166607293455014


вот так и работает катбуст. тоесть автоматически, не анализируя данные, прросто передал параметры и он предсказал  

# all feautures learning

In [19]:

X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price'] 

parameters = {"cat_features" : cat_features, 
             "eval_metric": "MAPE",
              "random_seed": 42,
             "verbose": 100}

лучше всегда фиксировать random_seed что бы отслеживать улучшения. так как в кат буст используются случайные числа.

есть там такой параметр random_seed и его можно передать или через словарь. или через параметр напрямую

In [20]:
# передаю параметры с помощью словаря. так как легко редактировать и уддалять параметры
model = CatBoostRegressor(**parameters)


In [21]:
model.fit(X=train[X], y=train[y], eval_set=(val[X], val[y]))

Learning rate set to 0.068263
0:	learn: 0.4607438	test: 0.4563750	best: 0.4563750 (0)	total: 2.51ms	remaining: 2.5s
100:	learn: 0.0904302	test: 0.0896388	best: 0.0896388 (100)	total: 165ms	remaining: 1.47s
200:	learn: 0.0781784	test: 0.0803125	best: 0.0803125 (200)	total: 288ms	remaining: 1.15s
300:	learn: 0.0729925	test: 0.0764506	best: 0.0764506 (300)	total: 424ms	remaining: 986ms
400:	learn: 0.0695552	test: 0.0744676	best: 0.0744553 (399)	total: 566ms	remaining: 846ms
500:	learn: 0.0670683	test: 0.0732827	best: 0.0732827 (500)	total: 710ms	remaining: 707ms
600:	learn: 0.0652835	test: 0.0726170	best: 0.0726128 (597)	total: 851ms	remaining: 565ms
700:	learn: 0.0636320	test: 0.0720622	best: 0.0720500 (695)	total: 993ms	remaining: 424ms
800:	learn: 0.0623408	test: 0.0714586	best: 0.0714329 (794)	total: 1.14s	remaining: 282ms
900:	learn: 0.0611487	test: 0.0711564	best: 0.0711321 (898)	total: 1.28s	remaining: 141ms
999:	learn: 0.0601407	test: 0.0708439	best: 0.0708403 (998)	total: 1.43s	r

получается катбуст смог сделать результат 7 процентов

In [22]:
model.predict(test[X])

array([57417.53972205, 22003.56027391, 13461.40876567, ...,
       30553.87789981, 13818.32099473, 24849.36058457])

In [23]:
test['price_pred_all'] = model.predict(test[X])

In [24]:
error(y_true=test['price'], y_pred=test['price_pred_all'])

1556.5378327680894
0.07367840910148211


# another parametrs

количество итераций и learning rate

настройка разных параметров и насторек

разработчики катбуст рекомнедуют, что бы точка переобучения, была ближе к концу 1000 итераций. у нас и так она на 998
bestIteration = 998

но если быдет раньше, можно попробовать покрутить learning rate - это отвечает за то, как быстро будет учиться модель. 

еще если модель быстро обучается, у катбуст есть такой параметр как early_stopping что бы не крутить итерации накоторых нет никакого улучшения

early_stopping_rounds=200

сокращает количество итераций после улучшения. если после улучшения 200 итераций не было нового улучшения, то модель останавливается

In [25]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price'] 

In [26]:

model = CatBoostRegressor(cat_features=cat_features, 
                          learning_rate=0.15,
                          eval_metric='MAPE', 
                          random_seed=42,
                         verbose=100,
                         )

In [27]:
model.fit(X=train[X], y=train[y], eval_set=(val[X], val[y]))

0:	learn: 0.4310189	test: 0.4270717	best: 0.4270717 (0)	total: 2.97ms	remaining: 2.97s
100:	learn: 0.0760459	test: 0.0788448	best: 0.0788448 (100)	total: 167ms	remaining: 1.49s
200:	learn: 0.0684429	test: 0.0738842	best: 0.0738842 (200)	total: 313ms	remaining: 1.24s
300:	learn: 0.0647135	test: 0.0725324	best: 0.0725099 (293)	total: 456ms	remaining: 1.06s
400:	learn: 0.0617826	test: 0.0716821	best: 0.0716821 (400)	total: 602ms	remaining: 900ms
500:	learn: 0.0593666	test: 0.0713216	best: 0.0711923 (496)	total: 748ms	remaining: 745ms
600:	learn: 0.0572029	test: 0.0711610	best: 0.0711393 (593)	total: 895ms	remaining: 594ms
700:	learn: 0.0554541	test: 0.0711559	best: 0.0710018 (695)	total: 1.04s	remaining: 445ms
800:	learn: 0.0539091	test: 0.0709705	best: 0.0709423 (765)	total: 1.19s	remaining: 295ms
900:	learn: 0.0523035	test: 0.0706992	best: 0.0706992 (900)	total: 1.33s	remaining: 147ms
999:	learn: 0.0511691	test: 0.0704407	best: 0.0704204 (984)	total: 1.48s	remaining: 0us

bestTest = 0.0

In [28]:


X = ['year', 'transmission','engineSize']

cat_features = ['transmission']

y = ['price'] 

model = CatBoostRegressor(cat_features=cat_features,
                          early_stopping_rounds=200,
                          eval_metric='MAPE',
                         random_seed=42,
                         verbose=100)

model.fit(X=train[X], y=train[y], eval_set=(val[X], val[y]))

Learning rate set to 0.068263
0:	learn: 0.4603375	test: 0.4555509	best: 0.4555509 (0)	total: 1.47ms	remaining: 1.46s
100:	learn: 0.1546393	test: 0.1529687	best: 0.1529687 (100)	total: 142ms	remaining: 1.26s
200:	learn: 0.1528213	test: 0.1517332	best: 0.1517332 (200)	total: 226ms	remaining: 900ms
300:	learn: 0.1517532	test: 0.1514386	best: 0.1514000 (288)	total: 325ms	remaining: 755ms
400:	learn: 0.1509722	test: 0.1515508	best: 0.1514000 (288)	total: 424ms	remaining: 633ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1514000297
bestIteration = 288

Shrink model to first 289 iterations.


скорость обучения регулируется early_stopping_rounds и learning_rate

# learning on all data

сейчас модель обучена на 60 процентах даннных.

60% трайн, 20% валидационные, 20% тестовые.


так как мы нашли парамтры когда модель дает лучший резльтат(номер итерации), можно дать модель больше данных

In [29]:
len(train), len(val), len(train) + len(val)

(6468, 2156, 8624)

In [30]:
train_full = pd.concat([train, val]) # соединяю трайн и валидационные

In [31]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']


parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'learning_rate': 0.08,
              'random_seed':42,
              'verbose':100}

model = CatBoostRegressor(**parameters)
model.fit(train[X],train[y],eval_set=(val[X],val[y]))


0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 5.24ms	remaining: 5.24s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 190ms	remaining: 1.69s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 330ms	remaining: 1.31s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 474ms	remaining: 1.1s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 631ms	remaining: 942ms
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 786ms	remaining: 783ms
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 936ms	remaining: 622ms
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 1.08s	remaining: 462ms
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 1.23s	remaining: 305ms
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 1.37s	remaining: 151ms
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 1.52s	remaining: 0us

bestTest = 0.06

In [32]:
# берем лучшую итерацию.
model.best_iteration_ + 1

994

In [33]:
# создаю параметры с лучшей итерацией
parameters = {'iterations': model.best_iteration_ + 1,
                'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'learning_rate': 0.08,
              'random_seed':42,
              'verbose':100}



In [112]:
model = CatBoostRegressor(**parameters)

In [34]:
# запускаю модель без валидационных данных
model.fit(train_full[X], train_full[y])

0:	learn: 0.4538016	total: 5.61ms	remaining: 5.61s
100:	learn: 0.0854178	total: 190ms	remaining: 1.69s
200:	learn: 0.0745022	total: 348ms	remaining: 1.38s
300:	learn: 0.0701717	total: 502ms	remaining: 1.17s
400:	learn: 0.0676577	total: 658ms	remaining: 982ms
500:	learn: 0.0656926	total: 810ms	remaining: 807ms
600:	learn: 0.0638104	total: 967ms	remaining: 642ms
700:	learn: 0.0624302	total: 1.12s	remaining: 479ms
800:	learn: 0.0612275	total: 1.28s	remaining: 319ms
900:	learn: 0.0599823	total: 1.44s	remaining: 159ms
999:	learn: 0.0589827	total: 1.6s	remaining: 0us


In [35]:
# модель обучилась, теперь делаю предикт на тесте
test['price_pred_all_features_and_data'] = model.predict(test[X])


In [36]:
error(y_true=test['price'], y_pred=test['price_pred_all'])

1556.5378327680894
0.07367840910148211


In [37]:
error(y_true=test['price'], y_pred=test['price_pred_all_features_and_data'])

1512.4065267188155
0.07305898308315235


прироста нет, но это лучше делать, так как чем больше данных тем лучше

# regression metrics

в этой задаче я использовал две метрики, средняя абсолютная ошибка - MAE

и средняя абсолютная ошибка в процентах - MAPE

есть еще и другие.
https://catboost.ai/en/docs/concepts/loss-functions-regression

в катбусте по умолчанию оптимизируется RMSE
loss_function по умолчанию RMSE
можно  поэкпеременитировать с метриками и посмотреть улучшиться ли результат

eval_metric используется для того что бы вовремя остановится на обучении

loss_function используется для того что бы идти к этой точке

In [38]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']




In [39]:
parameters = {'cat_features': cat_features,
              'loss_function': 'MAE',
              'eval_metric': 'MAPE',
              'learning_rate': 0.1,
              'random_seed':42,
              'verbose':100}

model = CatBoostRegressor(**parameters)


In [40]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.3849345	test: 0.3793341	best: 0.3793341 (0)	total: 2.87ms	remaining: 2.86s
100:	learn: 0.0764009	test: 0.0774578	best: 0.0774578 (100)	total: 191ms	remaining: 1.7s
200:	learn: 0.0666216	test: 0.0706498	best: 0.0706498 (200)	total: 365ms	remaining: 1.45s
300:	learn: 0.0628577	test: 0.0695086	best: 0.0694924 (298)	total: 537ms	remaining: 1.25s
400:	learn: 0.0605748	test: 0.0687890	best: 0.0687826 (399)	total: 707ms	remaining: 1.05s
500:	learn: 0.0589764	test: 0.0683467	best: 0.0683222 (490)	total: 874ms	remaining: 870ms
600:	learn: 0.0579201	test: 0.0680567	best: 0.0680567 (600)	total: 1.04s	remaining: 692ms
700:	learn: 0.0570454	test: 0.0678671	best: 0.0678659 (681)	total: 1.21s	remaining: 514ms
800:	learn: 0.0563072	test: 0.0676018	best: 0.0676018 (800)	total: 1.37s	remaining: 341ms
900:	learn: 0.0555798	test: 0.0674500	best: 0.0674384 (889)	total: 1.54s	remaining: 169ms
999:	learn: 0.0548863	test: 0.0673326	best: 0.0672972 (972)	total: 1.71s	remaining: 0us

bestTest = 0.06

разница не существенная. совсем чуток улучшили